In [61]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
import statsmodels.api as sm
from typing import List
import warnings
warnings.filterwarnings("ignore")


In [62]:
data = pd.read_stata("AER20090377_FinalData.dta")

data = data[data['year'] < 2004]
data = data[data['valid'] >= 9]
data = data[data['month'].isin([6, 7, 8])]


data = data[['fips', 'site_id', 'day', 'month', 'year']]
data.drop_duplicates(inplace=True)
data['NumDays'] = data.groupby(['fips', 'site_id', 'year'])['day'].transform('count')
data = data[data['NumDays'] >= 69]
data = data[['fips', 'site_id', 'year']]
data = data.drop_duplicates()
data = data.sort_values(by=['fips', 'site_id', 'year'])
data.to_stata('SummerList.dta', write_index=False)
del(data)

---

In [63]:
data = pd.read_stata("AER20090377_FinalData.dta")

data = data[data['valid'] >= 9]
data = data[[i for i in data.columns if '_merge' not in i]]
data = pd.merge(data, pd.read_stata('SummerList.dta'), how='inner', on=['fips', 'site_id', 'year'])
data.to_stata('temp.dta')

In [64]:
data.sort_values('fips',inplace=True)
data = pd.merge(data, pd.read_stata('AER20090377_NeighborData.dta'), on='fips', how='left', indicator=True)
data = data[data['_merge'] == 'left_only']
data.drop(columns=['_merge'], inplace=True)
data.loc[data.treat_CARB!=0, 'treat_rfg'] = 0
data.to_stata('temp.dta')

del(data)

---

Create weather variables to use in regressions---create interactions between months and weather vars


In [65]:
data = pd.read_stata('temp.dta')

# Day of week (DOW) and cross temperature effects
data['DOW'] = pd.to_datetime(data['Date']).dt.dayofweek

dow_dummies = pd.get_dummies(data['DOW'], prefix='_DM', drop_first=True)
for col in dow_dummies.columns:
    data[f'{col[:3]+col[4:]}'] = dow_dummies[col] * data['TempMax']

dow_dummies = pd.get_dummies(data['DOW'], prefix='_Dm', drop_first=True)
for col in dow_dummies.columns:
    data[f'{col[:3]+col[4:]}'] = dow_dummies[col] * data['TempMin']

dow_dummies = pd.get_dummies(data['DOW'], prefix='_Dr', drop_first=True)
for col in dow_dummies.columns:
    data[f'{col[:3]+col[4:]}'] = dow_dummies[col] * data['Rain']

dow_dummies = pd.get_dummies(data['DOW'], prefix='_Ds', drop_first=True)
for col in dow_dummies.columns:
    data[f'{col[:3]+col[4:]}'] = dow_dummies[col] * data['Snow']

# Day of Year (DOY)
data['DOY'] = pd.to_datetime(data['Date']).dt.dayofyear

# Temperature polynomials
data['TempMax1'] = data['TempMax']
data['TempMax2'] = data['TempMax']**2
data['TempMax3'] = data['TempMax']**3
data['TempMin1'] = data['TempMin']
data['TempMin2'] = data['TempMin']**2
data['TempMin3'] = data['TempMin']**3
data['TempMaxMin'] = data['TempMax'] * data['TempMin']
data['Rain1'] = data['Rain']
data['Rain2'] = data['Rain']**2
data['Snow1'] = data['Snow']
data['Snow2'] = data['Snow']**2
data['RainTempMax'] = data['Rain'] * data['TempMax']

In [66]:
# data.drop(columns=['level_0'], inplace=True)
data = data.sort_values(by=['fips', 'site_id', 'Date'])

data['TempMaxL1'] = data.groupby(['fips', 'site_id'])['TempMax'].shift(1)
data['TempMinL1'] = data.groupby(['fips', 'site_id'])['TempMin'].shift(1)
data['TempMaxL1'] = data['TempMaxL1'].fillna(np.nan)
data['TempMinL1'] = data['TempMinL1'].fillna(np.nan)

data['TempMaxMaxL1'] = data['TempMax'] * data['TempMaxL1']
data['TempMaxMinL1'] = data['TempMax'] * data['TempMinL1']

data['DOY'] = data['Date'].dt.dayofyear

temperature_vars = [i for i in data.columns if i>="TempMax1" and i <= "TempMaxMinL1"] 
for var in temperature_vars:
    data[f'DOY{var}'] = data['DOY'] * data[var]

data = data[data['Date'].dt.month.isin([6, 7, 8])]
data.to_stata('temp.dta')

---

Additional variables: income. Take logs of ozone

In [67]:
temp = pd.read_stata('temp.dta')
income_data = pd.read_stata('AER20090377_IncomeData.dta')

temp = temp.sort_values(by=['state_code', 'county_code', 'year'])
data = pd.merge(temp, income_data, on=['state_code', 'county_code', 'year'], how='inner', indicator=True)
data = data[data['_merge'] == 'both']
data = data.drop(columns=['_merge'])

data = data[(data['ozone_max'] != 0) & (data['epa_8hr'] != 0)]

data.dropna(subset=[i for i in data.columns if i>="TempMax1" and i <= "TempMaxMinL1"]+['income'], inplace=True)

data['lozone_max'] = np.log(data['ozone_max'])
data['lozone_8hr'] = np.log(data['epa_8hr'])
data.drop(columns=['treated_neighbor'],inplace=True)

---

Create census region dummies and interactions. Create time trend and interactions

In [68]:
data['region'] = None
data['division'] = None

region_map = {
    1:3,
    2:4,
    4:4,
    5:3,
    6:4,
    8:4,
    9:1,
    10:3,
    11:3,
    12:3,
    13:3,
    15:4,
    16:4,
    17:2,
    18:2,
    19:2,
    20:2,
    21:3,
    22:3,
    23:1,
    24:3,
    25:1,
    26:2,
    27:2,
    28:3,
    29:2,
    30:4,
    31:2,
    32:4,
    33:1,
    34:1,
    35:4,
    36:1,
    37:3,
    38:2,
    39:2,
    40:3,
    41:4,
    42:1,
    44:1,
    45:3,
    46:2,
    47:3,
    48:3,
    49:4,
    50:1,
    53:4,
    54:3,
    55:2,
    56:4
}

data['region'] = data['state_code'].map(region_map)

In [69]:
dow_dummies = pd.get_dummies(data[['year', 'region']], columns=['year', 'region'], prefix=['_RY', '_RY'], drop_first=True)
for col in dow_dummies.columns:
    data[f'{col[:3]+col[4:]}'] = dow_dummies[col]

dow_dummies = pd.get_dummies(data[['DOW', 'region']], columns=['DOW', 'region'], prefix=['_RW', '_RW'], drop_first=True)
for col in dow_dummies.columns:
    data[f'{col[:3]+col[4:]}'] = dow_dummies[col]

dow_dummies = pd.get_dummies(data[['region', 'DOY']], columns=['region', 'DOY'], prefix=['_RD', '_RD'], drop_first=True)
for col in dow_dummies.columns:
    data[f'{col[:3]+col[4:]}'] = dow_dummies[col]

data['DateS'] = (data["Date"] - pd.to_datetime("1960-01-01")).dt.days / 365
data['DateS2'] = data["DateS"] ** 2

In [70]:
data = data.drop(columns=['RVPCty', 'RFGCty', 'CARBCty'])

data['RVPCty'] = data.groupby('fips')['treat_rvpII'].transform('max')
data['RFGCty'] = data.groupby('fips')['treat_rfg'].transform('max')

data['RVPRFGCty'] = 0
data.loc[(data['RFGCty'] == 1) & (data['RVPCty'] == 1), 'RVPRFGCty'] = 1
data.loc[data['RVPRFGCty'] == 1, 'RFGCty'] = 0
data.loc[data['RVPRFGCty'] == 1, 'RVPCty'] = 0

data['CARBCty'] = data.groupby('fips')['treat_CARB'].transform('max')
data['CARBRFGCty'] = 0
data.loc[((data['RFGCty'] == 1) | (data['RVPRFGCty'] == 1)) & (data['CARBCty'] == 1), 'CARBRFGCty'] = 1

data.loc[data['CARBRFGCty'] == 1, 'RFGCty'] = 0
data.loc[data['CARBRFGCty'] == 1, 'RVPRFGCty'] = 0
data.loc[data['CARBRFGCty'] == 1, 'CARBCty'] = 0

In [71]:
for i in range(1, 5):
    col_name = f'TrendRVP{i}'
    data[col_name] = 0
    data.loc[(data['RVPCty'] == 1) & (data['region'] == i), col_name] = data['DateS']
    
    col_name = f'TrendRFG{i}'
    data[col_name] = 0
    data.loc[(data['RFGCty'] == 1) & (data['region'] == i), col_name] = data['DateS']
    
    col_name = f'TrendRVPRFG{i}'
    data[col_name] = 0
    data.loc[(data['RVPRFGCty'] == 1) & (data['region'] == i), col_name] = data['DateS']
    
    col_name = f'TrendCARB{i}'
    data[col_name] = 0
    data.loc[(data['CARBCty'] == 1) & (data['region'] == i), col_name] = data['DateS']
    
    col_name = f'TrendCARBRFG{i}'
    data[col_name] = 0
    data.loc[(data['CARBRFGCty'] == 1) & (data['region'] == i), col_name] = data['DateS']

In [72]:
for i in range(1, 5):
    data[f'QTrendRVP{i}'] = 0
    data.loc[(data['RVPCty'] == 1) & (data['region'] == i), f'QTrendRVP{i}'] = data['DateS']**2

    data[f'QTrendRFG{i}'] = 0
    data.loc[(data['RFGCty'] == 1) & (data['region'] == i), f'QTrendRFG{i}'] = data['DateS']**2

    data[f'QTrendRVPRFG{i}'] = 0
    data.loc[(data['RVPRFGCty'] == 1) & (data['region'] == i), f'QTrendRVPRFG{i}'] = data['DateS']**2

    data[f'QTrendCARB{i}'] = 0
    data.loc[(data['CARBCty'] == 1) & (data['region'] == i), f'QTrendCARB{i}'] = data['DateS']**2

    data[f'QTrendCARBRFG{i}'] = 0
    data.loc[(data['CARBRFGCty'] == 1) & (data['region'] == i), f'QTrendCARBRFG{i}'] = data['DateS']**2


In [73]:
data = data.sort_values(by=['state_code', 'year'])
data['StateYear'] = data.groupby(['state_code', 'year']).ngroup()
data.drop(columns=['level_0'],inplace=True)
data[[i for i in data.columns if i!="division"]].to_stata('DD_AnalysisDataset_NYR.dta')

---

First-difference everything on panelid

In [74]:
data = pd.read_stata('DD_AnalysisDataset_NYR.dta')

for var in ['fips', 'month', 'year', 'StateYear', 'state_code', 'EstTempFlag', 'EstTempFlagprcp']:
    data['A_'+var] = data[var]

vars_keep = ["DateS", "income", "panelid"]
for start in ["lozone_", "treat", "Temp", "Rain", "Snow",  "DOY", "Trend", "QTrend",  "_D", "_R",  "A"]:
    vars_keep += [col for col in data.columns if col.startswith(start)]

data.sort_values(by='panelid',inplace=True)

vars_to_process = ["DateS", "income"]
for start in ["lozone_", "treat", "Temp", "Rain", "Snow",  "DOY", "Trend", "QTrend",  "_D", "_R"]:
    vars_to_process += [col for col in data.columns if col.startswith(start)]

for var in vars_to_process:
    data[f'M{var}'] = data.groupby('panelid')[var].transform('mean')
    data[f'{var}D'] = data[var] - data[f'M{var}']
    data = data.drop(columns=[var, f'M{var}'])

data.drop(columns=['level_0']).to_stata("DD_AnalysisDataset_Diffed_NYR.dta")

del(data, vars_to_process, vars_keep)

---

Regressions

In [75]:
data = pd.read_stata('DD_AnalysisDataset_Diffed_NYR.dta')
data['incomeD'] = data['incomeD']/1000000000

Dependent var: daily max ozone

In [76]:
def genFormula(starts:List, fixed:List, y:str, nonconstant:bool = True) -> str:
    f = ""
    f += y + " ~ "
    x = fixed
    for start in starts:
        x += [col for col in data.columns if col.startswith(start)]
    f += ' + '.join(x)
    if nonconstant:
        f += " - 1"
    return f

def genVars(starts:List, fixed:List) -> str:
    x = fixed
    for start in starts:
        x += [col for col in data.columns if col.startswith(start)]
    return x

In [77]:
def model2txt(formula, model, outdir="DDResults_NYR.txt"):
    hypos = ['treat_rvpIID = treat_rfgD', 'treat_rvpIID = treat_CARBD', 'treat_rfgD = treat_CARBD']
    with open(outdir, "a") as f:
        f.write(formula + "\n\n\n")
        f.write(model.summary().as_text())
        for hypo in hypos:
            f.write('\n\n\n')
            f.write("Hypothesis: " + hypo)
            f.write('\n')
            f.write(model.t_test(hypo).summary().as_text())
        f.write("\n\n\n\n" + "#"*80 + '\n')

In [78]:
vars_all = genVars(['treat','Trend','QTrend','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD', 'lozone_maxD', 'A_StateYear'])
data = data.loc[~data[vars_all].isna().any(axis=1), vars_all]

In [79]:
formula = genFormula(['treat'], [], 'lozone_maxD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model)

formula = genFormula(['treat','_RY'], [], 'lozone_maxD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model)

formula = genFormula(['treat','_R'], [], 'lozone_maxD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model)

formula = genFormula(['treat','Temp','Rain','Snow','DOY','_D','_R'], [], 'lozone_maxD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model)

formula = genFormula(['treat','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD'], 'lozone_maxD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model)

formula = genFormula(['treat','Trend','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD'], 'lozone_maxD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model)

formula = genFormula(['treat','Trend','QTrend','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD'], 'lozone_maxD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model)

Dependent var: 8-hr ozone

In [80]:
data = pd.read_stata('DD_AnalysisDataset_Diffed_NYR.dta')
data['incomeD'] = data['incomeD']/1000000000
vars_all = genVars(['treat','Trend','QTrend','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD', 'lozone_8hrD', 'A_StateYear'])
data = data.loc[~data[vars_all].isna().any(axis=1), vars_all]

In [81]:
formula = genFormula(['treat'], [], 'lozone_8hrD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model, "DDResults_8hr_NYR.txt")

formula = genFormula(['treat','_RY'], [], 'lozone_8hrD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model, "DDResults_8hr_NYR.txt")

formula = genFormula(['treat','_R'], [], 'lozone_8hrD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model, "DDResults_8hr_NYR.txt")

formula = genFormula(['treat','Temp','Rain','Snow','DOY','_D','_R'], [], 'lozone_8hrD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model, "DDResults_8hr_NYR.txt")

formula = genFormula(['treat','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD'], 'lozone_8hrD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model, "DDResults_8hr_NYR.txt")

formula = genFormula(['treat','Trend','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD'], 'lozone_8hrD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model, "DDResults_8hr_NYR.txt")

formula = genFormula(['treat','Trend','QTrend','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD'], 'lozone_8hrD', True)
model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
model2txt(formula, model, "DDResults_8hr_NYR.txt")

---
Now DD regressions--split by urban, suburban, rural


In [82]:
for i in range(1, 4):
    data = pd.read_stata("DD_AnalysisDataset_NYR.dta")
    data = data[data['urban'] == i]

    for var in ['fips', 'month', 'year', 'StateYear', 'state_code', 'EstTempFlag', 'EstTempFlagprcp']:
        data['A_'+var] = data[var]

    vars_keep = ["DateS", "income", "panelid"]
    for start in ["lozone_", "treat", "Temp", "Rain", "Snow",  "DOY", "Trend", "QTrend",  "_D", "_R",  "A"]:
        vars_keep += [col for col in data.columns if col.startswith(start)]

    data.sort_values(by='panelid',inplace=True)

    vars_to_process = ["DateS", "income"]
    for start in ["lozone_", "treat", "Temp", "Rain", "Snow",  "DOY", "Trend", "QTrend",  "_D", "_R"]:
        vars_to_process += [col for col in data.columns if col.startswith(start)]

    for var in vars_to_process:
        data[f'M{var}'] = data.groupby('panelid')[var].transform('mean')
        data[f'{var}D'] = data[var] - data[f'M{var}']
        data = data.drop(columns=[var, f'M{var}'])

    data.drop(columns=['level_0']).to_stata(f"DD_AnalysisDataset_Diffed_NYR_{i}.dta")

del(data, vars_to_process, vars_keep)

Dependent var: daily max ozone. By urban/suburban/rural

In [83]:
for i in range(1, 4):
    data = pd.read_stata(f"DD_AnalysisDataset_Diffed_NYR_{i}.dta")
    data['incomeD'] = data['incomeD']/1000000000
    vars_all = genVars(['treat','Trend','QTrend','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD', 'lozone_maxD', 'A_StateYear'])
    data = data.loc[~data[vars_all].isna().any(axis=1), vars_all]
    
    formula = genFormula(['treat','Temp','Rain','Snow','DOY','_D','_R'], [], 'lozone_maxD', True)
    model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
    model2txt(formula, model, f"DDResults_NYR_{i}.txt")

    formula = genFormula(['treat','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD'], 'lozone_maxD', True)
    model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
    model2txt(formula, model, f"DDResults_NYR_{i}.txt")

    formula = genFormula(['treat','Trend','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD'], 'lozone_maxD', True)
    model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
    model2txt(formula, model, f"DDResults_NYR_{i}.txt")

    formula = genFormula(['treat','Trend','QTrend','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD'], 'lozone_maxD', True)
    model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
    model2txt(formula, model, f"DDResults_NYR_{i}.txt")

Dependent var: 8-hr ozone. By urban/suburban/rural

In [84]:
for i in range(1, 4):
    data = pd.read_stata(f"DD_AnalysisDataset_Diffed_NYR_{i}.dta")
    data['incomeD'] = data['incomeD']/1000000000
    vars_all = genVars(['treat','Trend','QTrend','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD', 'lozone_8hrD', 'A_StateYear'])
    data = data.loc[~data[vars_all].isna().any(axis=1), vars_all]
    
    formula = genFormula(['treat','Temp','Rain','Snow','DOY','_D','_R'], [], 'lozone_8hrD', True)
    model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
    model2txt(formula, model, f"DDResults_8hr_NYR_{i}.txt")

    formula = genFormula(['treat','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD'], 'lozone_8hrD', True)
    model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
    model2txt(formula, model, f"DDResults_8hr_NYR_{i}.txt")

    formula = genFormula(['treat','Trend','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD'], 'lozone_8hrD', True)
    model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
    model2txt(formula, model, f"DDResults_8hr_NYR_{i}.txt")

    formula = genFormula(['treat','Trend','QTrend','Temp','Rain','Snow','DOY','_D','_R'], ['incomeD'], 'lozone_8hrD', True)
    model = ols(formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['A_StateYear']})
    model2txt(formula, model, f"DDResults_8hr_NYR_{i}.txt")